In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
train = pd.read_csv('../input/rossmann-store-sales/train.csv', parse_dates = ['Date'], low_memory = False)
train.head()

In [ ]:
train['Date']=pd.to_datetime(train['Date'],format='%Y-%m-%d')
train['Date']

In [ ]:
train['Day'] = train['Date'].dt.day
train.drop('DayOfWeek', axis = 1, inplace = True)
train['DayOfWeek'] = train['Date'].dt.dayofweek
train['Month'] = train['Date'].dt.month
train['Year'] = train['Date'].dt.year
train['MonthYear'] = train['Date'].apply(lambda x : (str(x)[:7]))
train.head()

In [ ]:
train.info()

In [ ]:
test = pd.read_csv('../input/rossmann-store-sales/test.csv', parse_dates = True, low_memory = False)
test.head()

In [ ]:
test['Date']=pd.to_datetime(test['Date'],format='%Y-%m-%d')

In [ ]:
test['Day'] = test['Date'].dt.day
test.drop('DayOfWeek', axis = 1, inplace = True)
test['DayOfWeek'] = test['Date'].dt.dayofweek
test['Month'] = test['Date'].dt.month
test['Year'] = test['Date'].dt.year
test['MonthYear'] = test['Date'].apply(lambda x : (str(x)[:7]))
test

In [ ]:
test.drop(['Id'], axis = 1, inplace = True)
test.info()


In [ ]:
test['Open'].isnull().sum()

In [ ]:
test = test.drop(test[(test['Open'].isnull())].index)
test = test.reset_index(drop=True)


In [ ]:
test = test.astype({"Open": int})
test.info()

In [ ]:
#Drop closed stores as they have no sales.
print("Number of Closed stores: " ,np.sum([train['Open'] == 0]))
print("Number of No sales recorded: ", np.sum([train['Sales'] == 0]))
train = train.drop(train[(train.Open == 0) & (train.Sales == 0)].index)
train = train.reset_index(drop=True)
train = train.drop(['Open'],axis=1)
train

In [ ]:
train.max(axis = 0)

In [ ]:
#Remove NANs
#train = train[train['SalesPerCustomer'].notnull()] 

In [ ]:
store = pd.read_csv('../input/rossmann-store-sales/store.csv')
store.head(30)

In [ ]:
store.info()

In [ ]:
store.isnull().sum()

In [ ]:
store['CompetitionDistance'] = store['CompetitionDistance'].fillna(store['CompetitionDistance'].max())
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(store['CompetitionOpenSinceMonth'].mode().iloc[0]) #try 0
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear'].mode().iloc[0]) #try 0
store['Promo2SinceWeek'] = store['Promo2SinceWeek'].fillna(store['Promo2SinceWeek'].mode().iloc[0]) 
store['Promo2SinceYear'] = store['Promo2SinceYear'].fillna(store['Promo2SinceYear'].mode().iloc[0]) 
store['PromoInterval'] = store['PromoInterval'].fillna(store['PromoInterval'].mode().iloc[0]) 

store = store.astype({"CompetitionDistance": int, "CompetitionOpenSinceMonth": int, "CompetitionOpenSinceYear": int, "Promo2SinceWeek": int, "Promo2SinceYear": int})

store.info()

In [ ]:
df_store = store.merge(train, how='left', on='Store')

In [ ]:
df_store.max(axis = 0)

In [ ]:
df_store.info()

In [ ]:
df_store['StoreType'] = df_store['StoreType'].map({'a':1 , 'b':2 , 'c':3 , 'd':4})
df_store['StoreType'] = df_store['StoreType'].astype(int)
df_store['Assortment'] = df_store['Assortment'].map({'a':1 , 'b':2 , 'c':3})
df_store['Assortment'] = df_store['Assortment'].astype(int)
df_store['PromoInterval'] = df_store['PromoInterval'].map({'Jan,Apr,Jul,Oct':1 , 'Feb,May,Aug,Nov':2 , 'Mar,Jun,Sept,Dec':3})
df_store['PromoInterval'] = df_store['PromoInterval'].astype(int)
df_store['StateHoliday'] = df_store['StateHoliday'].map({'0':0 , 'a':1 , 'b':2 , 'c':3})
df_store['StateHoliday'] = df_store['StateHoliday'].astype(int)


In [ ]:
df_store.isnull().sum()

In [ ]:
df_store.info()

In [ ]:
df_store.drop('MonthYear', inplace = True, axis = 1)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_store.loc[[0]]

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

X = df_store.drop(['Store','Sales','Date','PromoInterval','Promo2SinceYear','Promo2SinceWeek','StateHoliday','SchoolHoliday'], axis = 1)
y = np.log1p(df_store['Sales'])

from sklearn.model_selection import train_test_split
X_train , X_val , y_train , y_val = train_test_split(X, y , test_size=0.30 , random_state = 1 )

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
X.head()

In [ ]:
def rmspe(y, yhat):
    rmspe = np.sqrt(np.mean( (y - yhat)**2 ))
    return rmspe

In [ ]:
#with the optimal parameters
rfr_val=RandomForestRegressor(n_estimators=72, 
                             criterion='mse', 
                             max_depth=20, 
                             min_samples_split=10, 
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, 
                             max_features='auto', 
                             max_leaf_nodes=None, 
                             min_impurity_decrease=0.0, 
                             min_impurity_split=None, 
                             bootstrap=True, 
                             oob_score=False,
                             n_jobs=4, 
                             random_state=35, 
                             verbose=0, 
                             warm_start=False)
model_RF_test=rfr_val.fit(X_train,y_train)

In [ ]:
yhat=model_RF_test.predict(X_val)

In [ ]:
plt.hist(yhat)

In [ ]:
error=rmspe(y_val,yhat)
error

In [ ]:
import itertools
importances = rfr_val.feature_importances_
std = np.std([rfr_val.feature_importances_ for tree in rfr_val.estimators_],
             axis=0)
indices = np.argsort(importances)
palette1 = itertools.cycle(sns.color_palette())

features_ranked=[]
for f in range(X_train.shape[1]):
    features_ranked.append(X_train.columns[indices[f]])


plt.figure(figsize=(10,15))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
            color=[next(palette1)], align="center")
plt.yticks(range(X_train.shape[1]), features_ranked)
plt.ylabel('Features')
plt.ylim([-1, X_train.shape[1]])
plt.show()

In [ ]:
df1 = pd.DataFrame({'Actual': y_val, 'Predicted': yhat})
df1.head(125)

In [ ]:
pred_sales = np.expm1(df1)
pred_sales

In [ ]:
import pickle
pickle.dump(model_RF_test, open('model2.pkl', 'wb'))